Соревнование на kaggle. Работа со свёрочными нейронными сетями (CNN).  
Ссылка на соревнование: https://www.kaggle.com/c/digit-recognizer  
Датасет содержит рукописные цифры. Предлагается обучить сеть распознавать цифры.  
Каждое изображение имеет размер 28х28 точек.  
Наиболее подходящий вариант для решения задачи - свёрточная нейронная сеть https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C  
В данной задаче рассматривается простая (не глубокая) свёрочная сеть. Для классификации более сложных изображений используются сети более сложной архитекуры.

In [93]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from keras.models import Sequential
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Для реализации CNN я буду использовать фреймворк Keras, он работает поверх Tensor Flow и позволяет строить сети любой сложности и размера.

In [94]:
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

In [95]:
labels=train_data['label']

In [96]:
train_data=train_data.drop(['label'],axis=1)

In [97]:
labels.value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

Как видно из выдачи - выборка практически равномерно распределена по классам.

In [98]:
labels = keras.utils.to_categorical(labels,len(labels.value_counts()))

In [99]:
train_data=train_data.astype('float32')/255

In [100]:
train_X,test_X,train_y,test_y=train_test_split(train_data,labels, random_state=123, test_size=0.2)

Я делю выборку на тестовую и тренировочную в пропорции 20/80 соответственно.

In [107]:
train_X=train_X.values.reshape(train_X.shape[0],28,28,1)
test_X=test_X.values.reshape(test_X.shape[0],28,28,1)

In [108]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [109]:
model.fit(train_X,train_y,batch_size=256,epochs=15,verbose=1,validation_data=(test_X, test_y))

Train on 33600 samples, validate on 8400 samples
Epoch 1/15
33600/33600 [==============================] - 7s 203us/step - loss: 0.5192 - acc: 0.8384 - val_loss: 0.1599 - val_acc: 0.9502
Epoch 2/15
33600/33600 [==============================] - 6s 184us/step - loss: 0.1460 - acc: 0.9586 - val_loss: 0.0894 - val_acc: 0.9713
Epoch 3/15
33600/33600 [==============================] - 6s 183us/step - loss: 0.1002 - acc: 0.9694 - val_loss: 0.0584 - val_acc: 0.9820
Epoch 4/15
33600/33600 [==============================] - 6s 185us/step - loss: 0.0809 - acc: 0.9750 - val_loss: 0.0573 - val_acc: 0.9832
Epoch 5/15
33600/33600 [==============================] - 6s 193us/step - loss: 0.0674 - acc: 0.9802 - val_loss: 0.0542 - val_acc: 0.9832
Epoch 6/15
33600/33600 [==============================] - 6s 191us/step - loss: 0.0586 - acc: 0.9822 - val_loss: 0.0445 - val_acc: 0.9862
Epoch 7/15
33600/33600 [==============================] - 6s 190us/step - loss: 0.0504 - acc: 0.9840 - val_loss: 0.0437 - v

По результатам обучения получилось точность 99.18% на тренировочной выборке и 98.89% на тестовой.  
Теперь можно построить прогноз по валидационной выборке на kaggle.

In [110]:
test_data=test_data.astype('float32')/255

In [111]:
test=test_data.values.reshape(len(test_data),28,28,1)

In [112]:
k=model1.predict_classes(test)
res=pd.DataFrame(data=k)
res.index+=1
res['ImageId']=res.index
res=res.rename(index=str, columns={0: "Label1"})
res['Label']=res['Label1']
res=res.drop(['Label1'],axis=1)

In [113]:
res.Label.value_counts()

1    3219
7    2881
2    2811
4    2796
0    2787
3    2780
6    2746
9    2744
8    2739
5    2497
Name: Label, dtype: int64

In [114]:
res.to_csv('result.csv', index=False)

По итогу валидации, точность на kaggle составила 98.60%.